# Neural Network

In [15]:
import torch.utils.data as tud
torch.utils.data

<module 'torch.utils.data' (namespace)>

<module 'torch.utils.data' (namespace)>

### loading libraries 

In [1]:
import pandas as pd                                         # to load data, data manipulation
import numpy as np                                          # for data manipulation
import matplotlib.pyplot as plt                             # for making plots, data visualization
import matplotlib.colors as colors                           
from sklearn.model_selection import train_test_split        # to split data into train and test sets
from sklearn.preprocessing import scale                     # to scale and center data

import torch
from torch import nn



from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset



ImportError: cannot import name 'DataLoader' from 'torch.utils.data' (unknown location)

In [93]:
device = 'cpu'

In [94]:
class NeuralNet(nn.Module):
    
    def __init__(self):
        
        super(NeuralNet, self).__init__()
        
        self.hidden_layer_1 = nn.Linear(33,64)
        self.hidden_activation = nn.ReLU()
        
        self.out = nn.Linear(64,1)
        
        
        
    def forward(self, x):
        
        x = self.hidden_layer_1(x)
        x = self.hidden_activation(x)
        
        x = self.out(x)
        
        return x

In [96]:
model = NeuralNet().to(device)

In [97]:
print(model)

NeuralNet(
  (hidden_layer_1): Linear(in_features=33, out_features=64, bias=True)
  (hidden_activation): ReLU()
  (out): Linear(in_features=64, out_features=1, bias=True)
)


In [98]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### importing UCI cervical cancer risk-factor data using pandas

In [42]:
! ls

Final presentation .pptx         assignment 02.pdf
GMM homework.docx                final project - NN.ipynb
GMM homework.pdf                 final project - SVM.ipynb
advcomp_assignment01.pdf         risk_factors_cervical_cancer.csv
assignment 02.docx


In [43]:
df = pd.read_csv('risk_factors_cervical_cancer.csv')

In [44]:
df.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0


In [45]:
df.dtypes

Age                                    int64
Number of sexual partners             object
First sexual intercourse              object
Num of pregnancies                    object
Smokes                                object
Smokes (years)                        object
Smokes (packs/year)                   object
Hormonal Contraceptives               object
Hormonal Contraceptives (years)       object
IUD                                   object
IUD (years)                           object
STDs                                  object
STDs (number)                         object
STDs:condylomatosis                   object
STDs:cervical condylomatosis          object
STDs:vaginal condylomatosis           object
STDs:vulvo-perineal condylomatosis    object
STDs:syphilis                         object
STDs:pelvic inflammatory disease      object
STDs:genital herpes                   object
STDs:molluscum contagiosum            object
STDs:AIDS                             object
STDs:HIV  

In [46]:
df['Age'].unique() # no missing values '?'

array([18, 15, 34, 52, 46, 42, 51, 26, 45, 44, 27, 43, 40, 41, 39, 37, 38,
       36, 35, 33, 31, 32, 30, 23, 28, 29, 20, 25, 21, 24, 22, 48, 19, 17,
       16, 14, 59, 79, 84, 47, 13, 70, 50, 49])

### data curation: missing data

The "STDs: Time since first diagnosis","STDs: Time since last diagnosis" columns are made up of mostly missing data. Let's exclude them from our dataset:

In [47]:
df = df.drop(columns=['STDs: Time since first diagnosis','STDs: Time since last diagnosis'])

In [48]:
df.loc[df['Number of sexual partners'] == '?', 'Number of sexual partners'] = 0
df.loc[df['First sexual intercourse'] == '?', 'First sexual intercourse'] = 0
df.loc[df['Num of pregnancies'] == '?', 'Num of pregnancies'] = 0
df.loc[df['Smokes'] == '?', 'Smokes'] = 0
df.loc[df['Smokes (years)'] == '?', 'Smokes (years)'] = 0
df.loc[df['Smokes (packs/year)'] == '?', 'Smokes (packs/year)'] = 0
df.loc[df['Hormonal Contraceptives'] == '?', 'Hormonal Contraceptives'] = 0
df.loc[df['Hormonal Contraceptives (years)'] == '?', 'Hormonal Contraceptives (years)'] = 0
df.loc[df['IUD'] == '?', 'IUD'] = 0
df.loc[df['IUD (years)'] == '?', 'IUD (years)'] = 0
df.loc[df['STDs'] == '?', 'STDs'] = 0
df.loc[df['STDs (number)'] == '?', 'STDs (number)'] = 0
df.loc[df['STDs:condylomatosis'] == '?', 'STDs:condylomatosis'] = 0
df.loc[df['STDs:cervical condylomatosis'] == '?', 'STDs:cervical condylomatosis'] = 0
df.loc[df['STDs:vaginal condylomatosis'] == '?', 'STDs:vaginal condylomatosis'] = 0
df.loc[df['STDs:vulvo-perineal condylomatosis'] == '?', 'STDs:vulvo-perineal condylomatosis'] = 0
df.loc[df['STDs:syphilis'] == '?', 'STDs:syphilis'] = 0
df.loc[df['STDs:pelvic inflammatory disease'] == '?', 'STDs:pelvic inflammatory disease'] = 0
df.loc[df['STDs:genital herpes'] == '?', 'STDs:genital herpes'] = 0
df.loc[df['STDs:molluscum contagiosum'] == '?', 'STDs:molluscum contagiosum'] = 0
df.loc[df['STDs:AIDS'] == '?', 'STDs:AIDS'] = 0
df.loc[df['STDs:HIV'] == '?', 'STDs:HIV'] = 0
df.loc[df['STDs:Hepatitis B'] == '?', 'STDs:Hepatitis B'] = 0
df.loc[df['STDs:HPV'] == '?', 'STDs:HPV'] = 0
df.loc[df['STDs: Number of diagnosis'] == '?', 'STDs: Number of diagnosis'] = 0
df.loc[df['Dx:Cancer'] == '?', 'Dx:Cancer'] = 0
df.loc[df['Dx:CIN'] == '?', 'Dx:CIN'] = 0
df.loc[df['Dx:HPV'] == '?', 'Dx:HPV'] = 0
df.loc[df['Dx'] == '?', 'Dx'] = 0
df.loc[df['Hinselmann'] == '?', 'Hinselmann'] = 0
df.loc[df['Schiller'] == '?', 'Schiller'] = 0
df.loc[df['Biopsy'] == '?', 'Biopsy'] = 0
df.loc[df['Citology'] == '?', 'Citology'] = 0

#avg_partners = df.loc[:, 'Number of sexual partners'].mean()
#avg_partners

In [49]:
df.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
2,34,1.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,0.0,0,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0


In [50]:
df.rename({'Number of sexual partners': 'N_PARTNERS'}, axis='columns', inplace=True)
df.rename({'First sexual intercourse': 'FIRST_INT'}, axis='columns', inplace=True)
df.rename({'Num of pregnancies': 'N_PREGNANCIES'}, axis='columns', inplace=True)
df.rename({'Smokes': 'SMOKER'}, axis='columns', inplace=True)
df.rename({'Smokes (years)': 'LEN_SMOKE'}, axis='columns', inplace=True)
df.rename({'Smokes (packs/year)':'AVG_SMOKE'}, axis='columns', inplace=True)
df.rename({'Hormonal Contraceptives': 'H_CONTRA'}, axis='columns', inplace=True)
df.rename({'Hormonal Contraceptives (years)': 'LEN_H_CONTRA'}, axis='columns', inplace=True)
df.rename({'IUD (years)': 'LEN_IUD'}, axis='columns', inplace=True)
df.rename({'STDs (number)': 'N_STDs'}, axis='columns', inplace=True)
df.rename({'STDs:condylomatosis': 'STD_CON'}, axis='columns', inplace=True)
df.rename({'STDs:cervical condylomatosis': 'STD_CC'}, axis='columns', inplace=True)
df.rename({'STDs:vaginal condylomatosis': 'STD_VC'}, axis='columns', inplace=True)
df.rename({'STDs:vulvo-perineal condylomatosis':'STD_VPC'}, axis='columns', inplace=True)
df.rename({'STDs:syphilis': 'STD_SYP'}, axis='columns', inplace=True)
df.rename({'STDs:pelvic inflammatory disease': 'STD_PID'}, axis='columns', inplace=True)
df.rename({'STDs:genital herpes': 'STD_GH'}, axis='columns', inplace=True)
df.rename({'STDs:molluscum contagiosum': 'STD_MC'}, axis='columns', inplace=True)
df.rename({'STDs:AIDS': 'STD_AIDS'}, axis='columns', inplace=True)
df.rename({'STDs:HIV': 'STD_HIV'}, axis='columns', inplace=True)
df.rename({'STDs:Hepatitis B': 'STD_HB'}, axis='columns', inplace=True)
df.rename({'STDs:HPV': 'STD_HPV'}, axis='columns', inplace=True)
df.rename({'STDs: Number of diagnosis': 'N_STD'}, axis='columns', inplace=True)
df.rename({'Dx:Cancer': 'CANCER'}, axis='columns', inplace=True)
df.rename({'Dx:CIN': 'CIN'}, axis='columns', inplace=True)
df.rename({'Dx:HPV': 'HPV'}, axis='columns', inplace=True)
df.rename({'Dx': 'DIAG'}, axis='columns', inplace=True)
df.rename({'Hinselmann': 'HINSELMANN'}, axis='columns', inplace=True)
df.rename({'Schiller': 'SCHILLER'}, axis='columns', inplace=True)
df.rename({'Biopsy': 'BIOPSY'}, axis='columns', inplace=True)
df.rename({'Citology': 'CITOLOGY'}, axis='columns', inplace=True)



In [51]:
df = df.astype('float')

In [52]:
df.head()

,Age,N_PARTNERS,FIRST_INT,N_PREGNANCIES,SMOKER,LEN_SMOKE,AVG_SMOKE,H_CONTRA,LEN_H_CONTRA,IUD,...,STD_HPV,N_STD,CANCER,CIN,HPV,DIAG,HINSELMANN,SCHILLER,CITOLOGY,BIOPSY
0,18.0,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52.0,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,46.0,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
df.dtypes

Age              float64
N_PARTNERS       float64
FIRST_INT        float64
N_PREGNANCIES    float64
SMOKER           float64
LEN_SMOKE        float64
AVG_SMOKE        float64
H_CONTRA         float64
LEN_H_CONTRA     float64
IUD              float64
LEN_IUD          float64
STDs             float64
N_STDs           float64
STD_CON          float64
STD_CC           float64
STD_VC           float64
STD_VPC          float64
STD_SYP          float64
STD_PID          float64
STD_GH           float64
STD_MC           float64
STD_AIDS         float64
STD_HIV          float64
STD_HB           float64
STD_HPV          float64
N_STD            float64
CANCER           float64
CIN              float64
HPV              float64
DIAG             float64
HINSELMANN       float64
SCHILLER         float64
CITOLOGY         float64
BIOPSY           float64
dtype: object

### splitting data into dependent and independent variables

In [54]:
X = df.drop('DIAG', axis=1).copy()
X.head()

,Age,N_PARTNERS,FIRST_INT,N_PREGNANCIES,SMOKER,LEN_SMOKE,AVG_SMOKE,H_CONTRA,LEN_H_CONTRA,IUD,...,STD_HB,STD_HPV,N_STD,CANCER,CIN,HPV,HINSELMANN,SCHILLER,CITOLOGY,BIOPSY
0,18.0,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52.0,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,46.0,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
y = df['DIAG'].copy()
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: DIAG, dtype: float64

### scaling the data

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train_scaled = scale(X_train)
X_test_scaled = scale(X_test)

In [90]:
X_train.shape

(643, 33)

In [82]:
y_test.to_numpy()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### data formating into torch

In [83]:
train_data_set = TensorDataset(torch.tensor(X_train_scaled, dtype=torch.float),
                              torch.tensor(y_train.to_numpy(), dtype=torch.float))

test_data_set = TensorDataset(torch.tensor(X_test_scaled, dtype=torch.float),
                              torch.tensor(y_test.to_numpy(), dtype=torch.float))

In [85]:
test_data_set

In [102]:
train_data_loader = DataLoader(train_data_set, batch_size = 20)
test_data_loader = DataLoader(test_data_set, batch_size = 20)

### putting it all together into a training function

In [104]:
def train(data_loader, nn_model, loss_fn, optimizer):
    
    nn_model.train()
    
    train_loss = 0
    
    for i,(X,y) in enumerate(data_loader):
        X,y = X.to(device), y.to(device)
        
        y_hat = nn_model(X)
        mse = loss_fn(y_hat, y)
        train_loss += mse.item() 
        
        optimizer.zero_grad()
        mse.backward()
        optimizer.step()
        
    num_batches = len(data_loader)
    avg_train_mse = train_loss / num_batches
    
    print(f'Train RMSE:{avg_train_mse**0.5}')

In [105]:
def test(data_loader, nn_model, loss_fn):
    
    nn_model.eval()
    
    test_loss = 0 
    
    with torch.no_grad():
        for X, y in data_loader:
            X,y = X.to(device), y.to(device)
            y_hat = model(X)
            test_loss += loss_fn(y_hat, y).item()
            
    num_batches = len(data_loader)
    avg_test_mse = test_loss / num_batches
    
    print(f'Test RMSE:{avg_test_mse**0.5}')

In [106]:
epochs = 10

for epoch in range(epochs):
    
    print(f'Epoch {epoch+1}:')
    
    train(train_data_loader, model, loss_fn, optimizer)
    test(test_data_loader, model, loss_fn)

Epoch 1:
Train RMSE:0.15903765574652068
Test RMSE:0.17800729400944246
Epoch 2:
Train RMSE:0.15958188764458145
Test RMSE:0.17773114429170941
Epoch 3:
Train RMSE:0.15940652217863527
Test RMSE:0.17734520458129815
Epoch 4:
Train RMSE:0.15976121552803463
Test RMSE:0.17686141398913346
Epoch 5:
Train RMSE:0.15971851578713397
Test RMSE:0.17661771501356022
Epoch 6:
Train RMSE:0.16004688511531598
Test RMSE:0.17643607683759158
Epoch 7:
Train RMSE:0.15973489190406662
Test RMSE:0.17636916857012624
Epoch 8:
Train RMSE:0.15982729762510195
Test RMSE:0.17649508349542395
Epoch 9:
Train RMSE:0.15938139713409766
Test RMSE:0.1765273469726146
Epoch 10:
Train RMSE:0.15983846899903134
Test RMSE:0.17688008642325545
